# Introduction

## **Team**: Diamonds
#### Francisco Arrieta, Lucia Camenisch, and Emily Schmidt

### Data Description

A company wants to assess the quality of their online advertisement campaign. Online users are the main interest in this campaign. The users see a web banner during their browsing activity. For each user, they want to be able to predict whether they subscribe to the advertised product through the advertisement banner, based on the information they have about them. To subscribe, the user has to click on the banner and then subscribe to the service.

* The target variable name: `subsscription`.

### Data Fields

**Unique Identifier**
* `Id`: A unique identifier of the observations in each dataset. In the test set, it is used to match your predictions with the true values.

**Target Variable** (only in the training data)
* `subscription`: whether the user subscribed through the banner (1: yes, 0: no)

**Demographic Variables**
* `age`: (numeric)

* `job`: Type of job (categorical: teacher, industrial_worker, entrepreneur, housekeeper, manager, retired, freelance, salesman, student, technology, unemployed, na)

* `marital`: marital status (categorical: married, divorced, single)

* `education`: (categorical: high_school, university, grad_school, na)

**Variables about the current campaign**
* `device`: From which device does the user see the banner? (categorical: smartphone, desktop, na)

* `day`: Last day of the month when the user saw the banner (numeric)

* `month`: Last month of the year when the user saw the banner (numeric)

* `time_spent`: How long the user looked at the banner last time (in seconds) (numeric)

* `banner_views`: Number of times the user saw the banner (numeric)

**Variables About an Old Campaign for the Same Product**
* `banner_views_old`: Number of times the user saw the banner during an old (and related) online ads campaign (numeric)

* `days_elapsed_old`: Number of days since the user saw the banner of an old (and related) online ads campaign (numeric, -1 if the user never saw the banner)

* `outcome_old`: Outcome of the old (and related) online ads campaign (categorical: failure, other, success, na)

**Variables with No Name**
* X1: (categorical: 1, 0)

* X2: (categorical: 1, 0)

* X3: (categorical: 1, 0)

* X4: (numeric)

## Project Structure

##### I Introduction
##### II Exploratory Analysis
##### III Data Imputation
##### IV Attempted Models
* LDA and QDA

* Logistic Regression

* k-Nearest Neighbors

* Support Vector Machines (SVM)

* Decision Tree

* Random Forest

* Neural Networks

* XGBoost

##### V  Model Comparison
* Model Selection Approach

#####  VI Final Model
* Best Predictive Model Description

* Tuning Parameters Analysis

##### VII Best Model Diagnostics and Final Kaggle Predicition

##### VIII Conclusion


